In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def add_headersToDf(dataframe):
    return dataframe.rename(columns={0: 'Chromossome', 1: 'First_index', 2: 'Last_index', 3: 'Citoband', 4: 'Unkown'})

In [3]:
def subtract_lastindex(dataframe):
    new_row = []
    for row in dataframe['Last_index']:
        new_row.append(row - 1)
    
    dataframe.drop('Last_index', axis = 1, inplace=True)

    dataframe.insert(2, "Last_index", new_row, True)

In [4]:
def add_sizeColumn(dataframe):
    sizes = []
    for row in range(len(dataframe.values)):
        size = dataframe["Last_index"][row] - dataframe["First_index"][row]
        sizes.append(size)
    dataframe.insert(len(dataframe.columns), "Size", sizes, True)

In [5]:
def getChromossome(df, n):
    """Returns pandas dataframe with only the citobands of one chromossome n
    """
    chomossome = str(n)
    chr_array = []
    #print("dataframe columns = ", df.columns)
    #chr_array.append(['Chromossome','First_index', 'Last_index', "Citoband", 'Unkown'])
    for row in df.values:
        cito_array = []
        if row[0] == chomossome:
            for col in row:
                cito_array.append(col)
            chr_array.append(cito_array)
    return pd.DataFrame(chr_array, columns=df.columns)#['Chromossome','First_index', 'Last_index', 'Citoband', 'Unkown', 'Size'] )

In [22]:
######################################## calc densities ############################################
def calc_STRDensity(cytobands_df, str_df):
    densities = []
    for chrom in cytobands_df["Chromossome"].unique():             
        print("searching cytobands in chromossome: ", chrom)
        chr_cytobands = cytobands_df.loc[cytobands_df['Chromossome'] == chrom]
        chr_size = chr_cytobands["Last_index"][chr_cytobands.index[-1]]+1
        cytobands_strs = np.zeros((chr_size), dtype=int)
        chr_strs = str_df.loc[str_df['FastaHeader'] == chrom]
        #percorrer o dataset pelas vulnerabilidades no cromossoma e anotar na lista np.zeroes
        for row in chr_strs.index:
            cytobands_strs[chr_strs["FirstIndex"][row]: chr_strs["LastIndex"][row]]+=1
        #Calculo da densidade para cada cytoband
        for cytoband in chr_cytobands.index:
            cytoband_array = cytobands_strs[int(chr_cytobands["First_index"][cytoband]):int(chr_cytobands["Last_index"][cytoband])+1]
            dr_length = np.count_nonzero(cytoband_array)
            cytoband_length = chr_cytobands["Size"][cytoband]
            density = dr_length/cytoband_length
            densities.append(round(density, 3))
        
    cytobands_df.insert(len(cytobands_df.columns), "STRs_density", densities, True)

In [7]:
def writetxt(txt_path, df):
    with open(txt_path, 'w') as f:
        df_string = df.to_string(header=False, index=False)
        f.write(df_string)
        f.close

def writeNewtxt(file_path, df):
    """Writes a pandas dataframe into txt, columns separated by tabs ("\t")
    """
    f = open(file_path, 'w')
    str_cols = ""
    for col in df.columns[:-1]:
        str_cols += col + "\t"
    str_cols += df.columns.values[-1]+ "\n"
    f.write(str(str_cols))
    for row in df.values:
        str_row = ""
        for col in row[:-1]:
            str_row += str(col) + "\t"
        str_row += str(row[-1])+ "\n"
        f.write(str_row)
    f.close

In [8]:
citoPath_read = "/home/androx/Documents/trabalho/citobands/cytobandFiltered_processed_complete.txt"
citoPath_write = "/home/androx/Documents/trabalho/citobands/cytobandFiltered_processed_str.txt"
#STRPath_read = "/home/androx/Documents/trabalho/datasets/STR/HomosapiensHG38_nov2014/HomosapiensHG38_FullGenome_processed.txt"
STRPath_read = "/home/androx/Documents/trabalho/datasets/STR/HomosapiensHG38_jan2016/HumanGenome_processed.txt"

citobands_df = add_headersToDf(pd.read_csv(citoPath_read, sep="\t"))#, header=None))
str_df = pd.read_csv(STRPath_read, sep="\t")
#print("columns: ", cito_dataframe.columns)

#subtract_lastindex(citobands_df)
#add_sizeColumn(citobands_df)
#print(citobands_df)

In [11]:
str_df

,Repeatid,FirstIndex,LastIndex,PatternSize,CopyNumber,ArrayLength,FastaHeader,FlankingLeft50,Pattern,FlankingRight50,Citoband
0,184820957,10001,10468,6,77.125000,468,chr1,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,TAACCC,cgcggtaccctcagccggcccgcccgcccgggtctgacctgaggag...,p36.33
1,184820958,10628,10800,29,5.965517,173,chr1,gcggtgctctccgggtctgtgctgaggagaacgcaactccgccgtt...,AGGCGCGCCGCGCCGGCGCAGGCGCAGAG,cacatgctagcgcgtcggggtggaggcgtggcgcaggcgcagagag...,p36.33
2,184820959,10823,10865,23,1.956522,43,chr1,aggcgcgccgcgccggcgcaggcgcagacacatgctagcgcgtcgg...,GAGGCGCGCCGCGCAGGCGCAGA,gcgcagagacacatgctaccgcgtccaggggtggaggcgtggcgca...,p36.33
3,184820960,10758,10997,76,3.210526,240,chr1,cgcagagaggcgcgccgcgccggcgcaggcgcagagaggcgcgccg...,GGCGCAGGCGCAGAGAGGCGCGCCGCGCCGGCGCAGGCGCAGAGAC...,cgcaagcctacgggcgggggttgggggggcgtgtgttgcaggagca...,p36.33
4,184820961,11226,11447,124,1.854839,222,chr1,cttgcagccgggcactacaggacccgcttgctcacggtgctgtgcc...,CGCCCCCTGCTGGCAACTAGGGAAACTGCAGGGCCCTCTTGCTCAA...,CCAACAGTACTGGCGGATTATAGGGAAACACCCGGAGCATATGCTG...,p36.33
...,...,...,...,...,...,...,...,...,...,...,...
1199623,186020580,57216407,57217320,6,150.222229,914,chrY,cctaatgcacacatgatacccaaaatataacacccacattcctcat...,TTAGGG,gtggtgtgtgggtgtgtgtgggtgtggtgtgtgtgggtgtggtgtg...,q12
1199624,186020581,57217317,57217414,6,16.375000,98,chrY,gttaggggttaggggttaggggttaggggttagggttagggttagg...,GGGTGT,TNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,q12
1199625,186020582,57217317,57217415,13,7.714286,99,chrY,gttaggggttaggggttaggggttaggggttagggttagggttagg...,GGGTGTGGTGTGT,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,q12
1199626,186020583,57217319,57217413,2,49.500000,95,chrY,taggggttaggggttaggggttaggggttagggttagggttagggt...,GT,gTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,q12


In [20]:
calc_STRDensity(citobands_df, str_df)
print(citobands_df)
#print(str_df)

searching cytobands in chromossome:  chr1
searching cytobands in chromossome:  chr10
searching cytobands in chromossome:  chr11
searching cytobands in chromossome:  chr12
searching cytobands in chromossome:  chr13
searching cytobands in chromossome:  chr14
searching cytobands in chromossome:  chr15
searching cytobands in chromossome:  chr16
searching cytobands in chromossome:  chr17
searching cytobands in chromossome:  chr18
searching cytobands in chromossome:  chr19
searching cytobands in chromossome:  chr2
searching cytobands in chromossome:  chr20
searching cytobands in chromossome:  chr21
searching cytobands in chromossome:  chr22
searching cytobands in chromossome:  chr3
searching cytobands in chromossome:  chr4
searching cytobands in chromossome:  chr5
searching cytobands in chromossome:  chr6
searching cytobands in chromossome:  chr7
searching cytobands in chromossome:  chr8
searching cytobands in chromossome:  chr9
searching cytobands in chromossome:  chrM
searching cytobands i

In [21]:
#writetxt(citoPath_write, citobands_df)
writeNewtxt(citoPath_write, citobands_df)